In [1]:
!pip install torch 

In [2]:
!pip install nltk

In [3]:
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import recall_score, precision_score, f1_score

import torch 
from torch import nn
from torch import optim

import datetime

In [4]:
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk import ngrams
from sklearn import metrics
import matplotlib.pyplot as plt
import pandas as pd

In [5]:
import nltk

nltk.download("wordnet")
nltk.download("punkt")
nltk.download('omw-1.4')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\agupt\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\agupt\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\agupt\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\agupt\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
# Load the training and validation data

train = pd.read_csv('../public_data/train/track_a/eng.csv')
val = pd.read_csv('../public_data/dev/track_a/eng_a.csv')

train.head()

,id,text,Anger,Fear,Joy,Sadness,Surprise
0,eng_train_track_a_00001,But not very happy.,0,0,1,1,0
1,eng_train_track_a_00002,Well she's not gon na last the whole song like...,0,0,1,0,0
2,eng_train_track_a_00003,She sat at her Papa's recliner sofa only to mo...,0,0,0,0,0
3,eng_train_track_a_00004,"Yes, the Oklahoma city bombing.",1,1,0,1,1
4,eng_train_track_a_00005,They were dancing to Bolero.,0,0,1,0,0


In [7]:
def pre_process(text):
    """ 
    Performs Different preprocessing operations.

    Parameters:
    text (string): passes a line of text (assume sentence segmentation has already been done)

    Returns:
    List[string]: Should return a list of tokens.
    """

    def separate_punctuation(text):
        text = re.sub(r"(\w)([.,;:!?'\"”\)])", r"\1 \2", text)
        text = re.sub(r"([.,;:!?'\"“\(\)])(\w)", r"\1 \2", text)
        return text

    def remove_punctuation(text):
        text = re.sub(r"(\w)([.,;:!?'\"”\)])", r"\1", text)
        text = re.sub(r"([.,;:!?'\"“\(\)])(\w)", r"\2", text)
        return text
        
    def tokenize_text(text):
        tokens = re.split(r"\s+",text)
        tokens = [t.lower() for t in tokens]
        return tokens

    def apply_stemming(tokens):
        stemmer = PorterStemmer()
        stemmed_tokens = [stemmer.stem(token) for token in tokens]
        return stemmed_tokens

    def apply_lemmatization(tokens):
        lemmatizer = WordNetLemmatizer()
        lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
        return lemmatized_tokens

    def generate_ngrams_from_tokens(tokens, n):
        return list(ngrams(tokens, n))


    # Separate Punctuation otherwise Remove it
    
    # text = separate_punctuation(text)
    text = remove_punctuation(text)
    
    # tokenize text
    
    tokens = tokenize_text(text)

    # Apply Lemmatization or Stemming

    # tokens = apply_stemming(tokens)
    tokens = apply_lemmatization(tokens)


    # Generate bigrams, trigrams and quadgrams
    bigrams = generate_ngrams_from_tokens(tokens, 2)
    bg = [i + " " + j for (i,j) in bigrams]
    tokens += bg

    # trigrams = generate_ngrams_from_tokens(tokens, 3)
    # tg = [i + " " + j + " " + k for (i,j,k) in trigrams]
    # tokens += tg

    # quadgrams = generate_ngrams_from_tokens(tokens, 4)
    # qg = [i + " " + j + " " + k + " " + l for (i,j,k,l) in quadgrams]
    # tokens += qg

    # Remove Stop words

    # stop_words = set(stopwords.words('english'))
    # tokens = [w for w in tokens if w not in stop_words]

    return "".join(tokens)

In [8]:
[pre_process(i) for i in train["text"]]

['butnotveryhappybut notnot veryvery happy',
 'wellshesnotgonnalastthewholesonglikethatsosinceimbehindherandtheaudiencecantseebelowmytorsoprettymuchiusemyhandtopushdownonthelidandsupportherweightwell shesshes notnot gongon nana lastlast thethe wholewhole songsong likelike thatthat soso sincesince imim behindbehind herher andand thethe audienceaudience cantcant seesee belowbelow mymy torsotorso prettypretty muchmuch ii useuse mymy handhand toto pushpush downdown onon thethe lidlid andand supportsupport herher weight',
 'shesatatherpapareclinersofaonlytomovenexttomeandstartclingingtomyarmshe satsat atat herher papapapa reclinerrecliner sofasofa onlyonly toto movemove nextnext toto meme andand startstart clingingclinging toto mymy arm',
 'yestheoklahomacitybombingyes thethe oklahomaoklahoma citycity bombing',
 'theyweredancingtobolerothey werewere dancingdancing toto bolero',
 'stillhadsexwithherthoughstill hadhad sexsex withwith herher though',
 'butiamexhausted-myeyefeelliketheyareabout

In [9]:
vectorizer = CountVectorizer(ngram_range=(1,2))
X_train = vectorizer.fit_transform([pre_process(i) for i in train["text"]]).toarray()
X_val = vectorizer.transform(val['text'].str.lower()).toarray()

emotions = ['Joy','Sadness','Surprise','Fear','Anger']
y_train = train[emotions].values
y_val = val[emotions].values

print(val)

                        id                                               text  \
0    eng_dev_track_a_00001            My mouth fell open `` No, no, no... I..   
1    eng_dev_track_a_00002  You can barely make out your daughter's pale f...   
2    eng_dev_track_a_00003  But after blinking my eyes for a few times lep...   
3    eng_dev_track_a_00004  Slowly rising to my feet I came to the conclus...   
4    eng_dev_track_a_00005  I noticed this months after moving in and doin...   
..                     ...                                                ...   
111  eng_dev_track_a_00112                       "ARcH stop your progression.   
112  eng_dev_track_a_00113        This 'star', starts to move across the sky.   
113  eng_dev_track_a_00114                                  and my feet hurt.   
114  eng_dev_track_a_00115        so i cried my eyes out and did the drawing.   
115  eng_dev_track_a_00116                              They were coal black.   

     Anger  Fear  Joy  Sadn

In [10]:
X_train_t = torch.Tensor(X_train)
y_train_t = torch.Tensor(y_train)

X_val_t = torch.Tensor(X_val)
y_val_t = torch.Tensor(y_val)

In [11]:
print(f'Shape of X: {X_train.shape}')
print(f'Shape of y: {y_train.shape}')
print(f'Number of positives per emotion class:')
_ = [print(f' - {e}: {v} ({round(100*v/len(y_train))}%)') for e,v in zip(emotions, y_train.sum(axis=0))]

Shape of X: (2768, 36386)
Shape of y: (2768, 5)
Number of positives per emotion class:
 - Joy: 674 (24%)
 - Sadness: 878 (32%)
 - Surprise: 839 (30%)
 - Fear: 1611 (58%)
 - Anger: 333 (12%)


In [12]:
model = nn.Sequential(
          nn.Linear(X_train.shape[1], 100),
          nn.ReLU(),
          nn.Dropout(0.3),
          nn.Linear(100, y_train.shape[1])
        )

In [13]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(model.parameters(), lr=1e-1, weight_decay=1e-2)

In [14]:
# Train for a set number of epochs
for epoch in range(1000):
    optimizer.zero_grad()
    output = model(X_train_t)
    loss = criterion(output, y_train_t)
    loss.backward()
    optimizer.step()
    if epoch % 100 == 0:
        print(f'Epoch {epoch}: Loss: {round(loss.item(),3)}')

Epoch 0: Loss: 0.695
Epoch 100: Loss: 0.589
Epoch 200: Loss: 0.571
Epoch 300: Loss: 0.567
Epoch 400: Loss: 0.565
Epoch 500: Loss: 0.562
Epoch 600: Loss: 0.561
Epoch 700: Loss: 0.56
Epoch 800: Loss: 0.558
Epoch 900: Loss: 0.557


In [15]:
def get_predictions(X_val, model, threshold=0.5):
    sig = nn.Sigmoid() 
    yhat = sig(model(X_val)).detach().numpy()
    y_pred = yhat > threshold
    
    return y_pred

In [29]:
y_pred = get_predictions(X_val_t, model, 0.45)
# print(y_pred)

# Create a DataFrame to save to CSV
val_data_with_pred = pd.DataFrame(y_pred, columns=['Anger', 'Fear', 'Joy', 'Sadness', 'Surprise'])  # Adjust column names as per your features
# val_data_with_pred['True_Label'] = y_test
# val_data_with_pred['Predictions'] = dummy_predictions

val_data_with_pred = val_data_with_pred.astype(int)

val_data_with_pred['id'] = val['id']

val_data_with_pred = val_data_with_pred[['id', 'Anger', 'Fear', 'Joy', 'Sadness', 'Surprise']]

# Save to CSV
current_time = datetime.datetime.now()
formatted_time = current_time.strftime('%Y-%m-%d %H:%M:%S')

# val_data_with_pred.to_csv(f'results/pred_eng_a_{formatted_time}.csv', index=False)

print(val_data_with_pred)

                        id  Anger  Fear  Joy  Sadness  Surprise
0    eng_dev_track_a_00001      1     1    1        1         0
1    eng_dev_track_a_00002      1     1    1        1         0
2    eng_dev_track_a_00003      1     1    1        1         0
3    eng_dev_track_a_00004      1     1    1        1         0
4    eng_dev_track_a_00005      1     1    1        1         0
..                     ...    ...   ...  ...      ...       ...
111  eng_dev_track_a_00112      1     1    1        1         0
112  eng_dev_track_a_00113      1     1    1        1         0
113  eng_dev_track_a_00114      1     1    1        1         0
114  eng_dev_track_a_00115      1     1    1        1         0
115  eng_dev_track_a_00116      1     1    1        1         0

[116 rows x 6 columns]


In [30]:
def evaluate(y_val, y_pred):
    for average in ['micro', 'macro']:
        recall = recall_score(y_val, y_pred, average=average, zero_division=0)
        precision = precision_score(y_val, y_pred, average=average, zero_division=0)
        f1 = f1_score(y_val, y_pred, average=average, zero_division=0)
    
        print(f'{average.upper()} recall: {round(recall, 4)}, precision: {round(precision, 4)}, f1: {round(f1, 4)}')

In [31]:
# evaluate(y_val, y_pred) EVALS WON'T WORK HERE 

In [32]:
def evaluate_per_class(y_val, y_pred):
    for i, emotion in enumerate(emotions):
        print(f'*** {emotion} ***')
    
        recall = recall_score(y_val[:,i], y_pred[:,i], zero_division=0)
        precision = precision_score(y_val[:,i], y_pred[:,i], zero_division=0)
        f1 = f1_score(y_val[:,i], y_pred[:,i], zero_division=0)
        
        print(f'recall: {round(recall, 4)}, precision: {round(precision, 4)}, f1: {round(f1, 4)}\n')

In [33]:
# evaluate_per_class(y_val, y_pred) EVALS WON'T WORK HERE 

In [34]:
weights = y_train.sum(axis=0)/y_train.sum()
weights = max(weights)/weights

In [35]:
# Define model 
model = nn.Sequential(
          nn.Linear(X_train.shape[1], 100),
          nn.ReLU(),
          nn.Dropout(0.3),
          nn.Linear(100, y_train.shape[1])
        )

# Define training parameters
criterion = nn.BCEWithLogitsLoss(pos_weight=torch.Tensor(weights)) # <-- weights assigned to optimiser
optimizer = optim.SGD(model.parameters(), lr=1e-1, weight_decay=1e-2)

# Train for a number of epochs
for epoch in range(1000):
    optimizer.zero_grad()
    output = model(X_train_t)
    loss = criterion(output, y_train_t)
    loss.backward()
    optimizer.step()
    if epoch % 100 == 0:
        print(f'Epoch {epoch}: Loss: {round(loss.item(),3)}')

# Get predictions
y_pred = get_predictions(X_val_t, model, 0.45)

# Create a DataFrame to save to CSV
val_data_with_pred = pd.DataFrame(y_pred, columns=['Anger', 'Fear', 'Joy', 'Sadness', 'Surprise'])  # Adjust column names as per your features
# val_data_with_pred['True_Label'] = y_test
# val_data_with_pred['Predictions'] = dummy_predictions

val_data_with_pred = val_data_with_pred.astype(int)

val_data_with_pred['id'] = val['id']

val_data_with_pred = val_data_with_pred[['id', 'Anger', 'Fear', 'Joy', 'Sadness', 'Surprise']]

# Save to CSV
# current_time = datetime.datetime.now()
# formatted_time = current_time.strftime('%Y-%m-%d %H:%M:%S')

# val_data_with_pred.to_csv(f'../results/pred_eng_a_{formatted_time}.csv', index=False)

print(val_data_with_pred)

# Evaluate: EVALS WON'T WORK HERE 
# print('\n\nEVALUATION\n')
# evaluate(y_val, y_pred)

# print('\nPER CLASS BREAKDOWN\n')
# evaluate_per_class(y_val, y_pred)

Epoch 0: Loss: 0.877
Epoch 100: Loss: 0.863
Epoch 200: Loss: 0.857
Epoch 300: Loss: 0.852
Epoch 400: Loss: 0.847
Epoch 500: Loss: 0.843
Epoch 600: Loss: 0.837
Epoch 700: Loss: 0.832
Epoch 800: Loss: 0.826
Epoch 900: Loss: 0.82


OSError: [Errno 22] Invalid argument: '../results/pred_eng_a.csv_2024-12-13 13:06:21'

In [36]:
val_data_with_pred

,id,Anger,Fear,Joy,Sadness,Surprise
0,eng_dev_track_a_00001,0,1,1,1,0
1,eng_dev_track_a_00002,0,1,1,1,0
2,eng_dev_track_a_00003,0,1,1,1,0
3,eng_dev_track_a_00004,0,1,1,1,0
4,eng_dev_track_a_00005,0,1,1,1,0
...,...,...,...,...,...,...
111,eng_dev_track_a_00112,0,1,1,1,0
112,eng_dev_track_a_00113,0,0,1,1,0
113,eng_dev_track_a_00114,0,1,1,1,0
114,eng_dev_track_a_00115,0,1,1,1,0
